In [1]:
import os
os.chdir("../function")

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import load_data
import Diagnosis_DF
import multiprocessing
from mapping import generate_mapping_function
from mapping_dict import get_breed_dict

In [3]:
def to_date(x):
    try:
        x = x.astype(int)
    except:
        pass
    return x

In [4]:
diagnosis = pd.read_pickle("../File/Diagnosis.pkl")
df_dict = pd.read_excel("../File/통합용어정리.xlsx")
df_dict2 = df_dict.copy()
df_dict

,분류,대분류,Keyword,동의어1,동의어2,동의어3,동의어4,동의어5,동의어6,동의어7,동의어8,동의어9,동의어10,동의어11,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,ㄱ,안과,각막염,각막궤양,각막 궤양,각막 염,각막염,corneal ulcer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ㄱ,신경계,간질,간질,뇌전증,epilepsy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ㄱ,내분비,갑상선기능저하증,hypothyroidism,갑상선기능저하,갑상선 기능저하,갑상선 기능 저하,euthyroid sick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ㄱ,귀/피부,개선충증,개옴,개선중증,옴진드기,옴 진드기,scabie,sarcoptic mange,개선충,옴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ㄱ,일반,건강검진,검진,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,E,귀/피부,Ehrlichiosis,에르리키아,에르리히,ehrlichi,에르키아,에를리히,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,G,소화기,Giardia,지알,giar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,I,감염성,Influenza,인플루엔자,influenza,독감,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,M,귀/피부,Malasseziasis,말라세지아,말라세시아,말라세치아,말라시지아,말라쎄지아,말라쌔지아,말라쎼지아,malassezia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_dict.fillna("NA",inplace=True)
dict_map = {}
for i,q in df_dict.iterrows():
    temp = q[2:].values.tolist()
    temp = [x for x in temp if x != "NA"]
    dict_map[q["Keyword"]] = temp

def mapping(x):
    y = []
    for i,v in dict_map.items():
        for vv in v:
            if vv.lower() in x["Diagnosis"].lower():
                y.append(i)
            else:
                continue
    if len(y) > 0:
        return y
    else:
        return x["Diagnosis"]

    
    
    
def diag_mapping(x):
    y = []
    for i,v in dict_map.items():
        for z in v:
            try :
                if z.lower() in x["Diagnosis"].lower():
                    y.append(i)
                else:
                    continue
            except :
                pass
        #중복 제거
        y_set = set(y)  # 집합set으로 변환
        y = list(y_set)    
            
    if len(y) > 0:
        return y
    else:
        return x["Diagnosis"]
tqdm.pandas()
diagnosis["Diagnosis_named"] = diagnosis.progress_apply(lambda x : mapping(x),axis=1)
diagnosis

100%|██████████| 1088454/1088454 [28:56<00:00, 626.70it/s]


,Name2,Name3,Sex,Diagnosis,_Month,_Year,Diagnosis_named
0,Cocker Spaniel,코커 스패니얼,4,호산성 간염,46,3,호산성 간염
1,Cocker Spaniel,코커 스패니얼,4,외이염,46,3,[이염(외이염)]
2,Cocker Spaniel,코커 스패니얼,4,흑색가시세포증,22,1,흑색가시세포증
3,Cocker Spaniel,코커 스패니얼,4,호산성 간염,22,1,호산성 간염
4,Cocker Spaniel,코커 스패니얼,4,외이염,22,1,[이염(외이염)]
...,...,...,...,...,...,...,...
131,Greyhound,그레이하운드,0,중성화수술,87,7,"[중성화, 중성화]"
132,Poodle,푸들,3,중성화수술,5,0,"[중성화, 중성화]"
133,Poodle,푸들,4,경련,51,4,"[경련, 경련]"
134,Poodle,푸들,3,유치발치,11,0,"[발치, 발치, 발치]"


In [6]:
temp = pd.DataFrame(columns = diagnosis.columns)
temp2 = pd.DataFrame(columns = diagnosis.columns)
for i,q in tqdm(diagnosis.iterrows(),total = len(diagnosis)):
    if type(q["Diagnosis_named"]) !=str:
        for qq in q["Diagnosis_named"]:
            y = pd.DataFrame(q)
            y = y.T
            y["Diagnosis_named"] = qq
            temp = pd.concat([temp,y])
   
    else:
        y = pd.DataFrame(q)
        y = y.T
        temp2 = pd.concat([temp2,y])

100%|██████████| 1088454/1088454 [31:45:35<00:00,  9.52it/s]   


In [7]:
diagnosis_named = pd.concat([temp,temp2])
diagnosis_named.rename(columns = {"Diagnosis":"Original","Diagnosis_named":"Diagnosis"},inplace=True)

In [8]:
diagnosis_named

,Name2,Name3,Sex,Original,_Month,_Year,Diagnosis
1,Cocker Spaniel,코커 스패니얼,4,외이염,46,3,이염(외이염)
4,Cocker Spaniel,코커 스패니얼,4,외이염,22,1,이염(외이염)
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,발치
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,발치
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,스케일링
...,...,...,...,...,...,...,...
11,코카스파니엘,,3,봉와직염,125,10,봉와직염
24,Munchkin,먼치킨,3,장절개수술 이물제거,30,2,장절개수술 이물제거
78,Boston Terrier,보스턴 테리어,4,봉합,103,8,봉합
95,Maltese,말티즈,4,안검 맥립종 제거,84,7,안검 맥립종 제거


In [10]:
mapping_dict = get_breed_dict()

tqdm.pandas()
diagnosis_named["Name"] = diagnosis_named.progress_apply(lambda row : generate_mapping_function(row,mapping_dict),axis=1)

100%|██████████| 1861360/1861360 [02:29<00:00, 12482.74it/s]
<ipython-input-10-20c290cf6e8f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diagnosis_named["Name"] = diagnosis_named.progress_apply(lambda row : generate_mapping_function(row,mapping_dict),axis=1)


In [11]:
diagnosis_named

,Name2,Name3,Sex,Original,_Month,_Year,Diagnosis,Name
1,Cocker Spaniel,코커 스패니얼,4,외이염,46,3,이염(외이염),Cocker_Spaniel
4,Cocker Spaniel,코커 스패니얼,4,외이염,22,1,이염(외이염),Cocker_Spaniel
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,발치,Maltese
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,발치,Maltese
5,Maltese,몰티즈,3,스케일링 및 발치,132,11,스케일링,Maltese
...,...,...,...,...,...,...,...,...
11,코카스파니엘,,3,봉와직염,125,10,봉와직염,Cocker_Spaniel
24,Munchkin,먼치킨,3,장절개수술 이물제거,30,2,장절개수술 이물제거,Unknown
78,Boston Terrier,보스턴 테리어,4,봉합,103,8,봉합,Unknown
95,Maltese,말티즈,4,안검 맥립종 제거,84,7,안검 맥립종 제거,Maltese


In [12]:
diagnosis_named = diagnosis_named[(diagnosis_named["Sex"].isin([1,2,3,4])) & (diagnosis_named["_Month"] > 0)]
diagnosis_named[diagnosis_named["Name"].isnull()]
diagnosis_named = diagnosis_named[diagnosis_named["Name"] != "Unknown"]

In [13]:
diag_map = {}
for i,q in df_dict2.iterrows():
    diag_map[q["Keyword"]] = q["대분류"]
diagnosis_named["뿜빵이"] = diagnosis_named["Diagnosis"].map(diag_map)

In [14]:
diagnosis_named.to_csv("../File/Diagnosis_named.csv",index=False)